<a href="https://colab.research.google.com/github/Sapiosonic/hackeando-bolsa-varos/blob/main/aula_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Projeto 1 - Backtest modelo de investimento Magic Formula.


### Desafio:

* Testar se a regra de investimento da fórmula mágica, do Joel Grenblatt, funcionou no Brasil nos últimos anos.


### Passo a passo:

   **Passo 1** - Importar os módulos e bibliotecas.
   
   **Passo 2** - Baixar os dados disponibilizados.
   
   **Passo 3** - Calcular os retornos mensais das empresas
   
   **Passo 4** - Filtrar  a liquidez.
   
   **Passo 5** - Criar o ranking dos indicadores.
   
   **Passo 6** - Criar as carteiras.
   
   **Passo 7** - Calcular a rentabilidade por carteira.
   
   **Passo 8** - Calcular a rentabilidade do modelo.

   **Passo 9** - Calcular a rentabilidade do ibovespa no mesmo período.

   
   ##### **Passo 10** - Analisar os resultados.
   
-------------------

# Passo 1: Importando os módulos necessários

In [ ]:
#!pip install quantstats

In [ ]:
import pandas as pd
import quantstats as qs

# Passo 2: Baixar os dados dispobinilizados.

In [ ]:
dados_empresas = pd.read_csv("dados_empresas.csv")

# Passo 3: Calcular os retornos mensais das empresas.

In [ ]:
dados_empresas['retorno'] = dados_empresas.groupby('ticker')['preco_fechamento_ajustado'].pct_change()
dados_empresas['retorno'] = dados_empresas.groupby('ticker')['retorno'].shift(-1)

# Passo 4: Filtrar a liquidez.

In [ ]:
dados_empresas = dados_empresas[dados_empresas['volume_negociado'] > 1000000]

# Passo 5: Criar o ranking dos indicadores.

In [ ]:
dados_empresas['ranking_ev_ebit'] = dados_empresas.groupby('data')['ebit_ev'].rank(ascending = False)
dados_empresas['ranking_roic'] = dados_empresas.groupby('data')['roic'].rank(ascending = False)

dados_empresas['ranking_final'] = dados_empresas['ranking_ev_ebit'] + dados_empresas['ranking_roic']
dados_empresas['ranking_final'] = dados_empresas.groupby('data')['ranking_final'].rank()

In [ ]:
dados_empresas[dados_empresas['data'] == '2017-06-30'].sort_values('ranking_final').head(20)

# Passo 6: Criar  as carteiras.

In [ ]:
dados_empresas = dados_empresas[dados_empresas['ranking_final'] <= 10]

# Passo 7: Calcular a rentabilidade por carteira.

In [ ]:
rentabilidade_por_carteiras = dados_empresas.groupby('data')['retorno'].mean()
rentabilidade_por_carteiras = rentabilidade_por_carteiras.to_frame()

# Passo 8: Calcular a rentabilidade do modelo.

In [ ]:
rentabilidade_por_carteiras['Magic Formula'] = (rentabilidade_por_carteiras['retorno'] + 1).cumprod() - 1

rentabilidade_por_carteiras = rentabilidade_por_carteiras.shift(1)

rentabilidade_por_carteiras = rentabilidade_por_carteiras.dropna()

# Passo 9: Calcular a rentabilidade do ibovespa no mesmo período.

In [ ]:
ibov = pd.read_csv('ibov.csv')

retornos_ibov = ibov['fechamento'].pct_change().dropna()
retorno_acum_ibov = (1 + retornos_ibov).cumprod() - 1
rentabilidade_por_carteiras['Ibovespa'] = retorno_acum_ibov.values
rentabilidade_por_carteiras = rentabilidade_por_carteiras.drop('retorno', axis = 1)

# Passo 10: Analisar os resultados.

In [ ]:
qs.extend_pandas()
rentabilidade_por_carteiras.index = pd.to_datetime(rentabilidade_por_carteiras.index)

rentabilidade_por_carteiras['Magic Formula'].plot_monthly_heatmap()
rentabilidade_por_carteiras['Ibovespa'].plot_monthly_heatmap()

In [ ]:
rentabilidade_por_carteiras.plot(figsize = (7, 4))